### [ENCODE](https://www.encodeproject.org/)数据收集和预处理

- 表观数据  
  ATAC, H3K27ac, H3K27me3, H3K36me3, H3K4me1, H3K4me3, H3K9me3  
- 细胞系
  选择ENCODE数据库中有以上表观数据的细胞系，筛选后包括：  
  K562, HepG2, GM12878, MCF-7, SK-N-SH, HCT116, IMR-90  
- 转录因子
  以上细胞系涵盖的转录因子并且在至少4个细胞系中被测到，包含两类，`1)`数据库如[Jaspar](https://jaspar.elixir.no/)等中有相关基序记录，`2)`数据库中没有基序记录。前者可直接结合DNA，后者可能通过co-factor结合DNA。其中：

  I类（33个）包括：'ATF3', 'BHLHE40', 'CEBPB', 'CTCF', 'EGR1', 'ELF1', 'ELK1', 'FOXM1', 'GABPA', 'JUND', 'MAFK', 'MYC', 'NRF1', 'REST', 'RFX5', 'SP1', 'SRF', 'TCF7L2', 'USF1', 'USF2', 'YY1', 'ZBTB33', 'ZBTB40', 'ZNF24', 'ZNF274', 'CREB1', 'FOS', 'FOXK2', 'MAX', 'MAZ', 'MXI1', 'TCF12', 'TEAD4',

  II类（11个）包括：'CHD1', 'CHD2', 'EP300', 'HDAC2', 'POLR2A', 'RAD21', 'RCOR1', 'SIN3A', 'SMC3', 'TAF1', 'TARDBP',

In [ ]:
import time
from pathlib import Path

import numpy as np
import pandas as pd

import requests
import json

#### ENCODE-Experiment report记录表及文件下载

时间：2023-05-08  
物种： 人类  
文件：  
1. 原始记录，从ENCODE直接下载，experiment_report_2023_5_8_3h_3m.tsv  
2. 人类（Homo sapiens）数据，experiment_report_human_keep.tsv  
3. 结果文件，带星标，released_files_of_assays_default_samples.csv
4. 

In [ ]:
info_file='experiment_report_2023_5_8_3h_3m.tsv'
info=pd.read_table(info_file,skiprows=1)

#选择数据
assay_keep=['ATAC-seq', 'ChIP-seq']
target_keep=['H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K36me3', 'H3K27ac','nan']

info_human=info[info['Organism']=='Homo sapiens']
info_human_fill=info_human.fillna('nan')
info_human_keep=info_human_fill[info_human_fill['Assay name'].isin(assay_keep)]
info_human_keep=info_human_keep[info_human_keep['Target of assay'].isin(target_keep)]

info_file_human_keep='experiment_report_human_keep.tsv'
info_human_keep.to_csv(info_file_human_keep,index=False,sep='\t')

In [ ]:
#函数，获取实验详细信息，包括结果文件等
#
url_org='https://www.encodeproject.org'

def searcher(experiment_id,experiment_assay,target='uknown',biosample='uknown'):
    #根据实验ID搜索实验相关数据文件，区别peak文件（bed格式）和rna水平文件（tsv格式）
    if experiment_assay in ['ATAC-seq','DNase-seq','ChIP-seq']:
        file_format='bed'
        output_type='peaks&output_type=pseudoreplicated+peaks&output_type=replicated+peaks'
    elif experiment_assay=='RNA-seq':
        file_format='tsv'
        output_type='gene+quantifications'
    else:
        print("experiment type wrong, one of [ATAC_seq, DNase-seq, ChIP-seq, RNA-seq]")
        print("##### accession id wrong", experiment_id)
    #定制搜索url（REST API）
    url=url_org+'/search/?type=File&dataset=/experiments/%s/&file_format=%s&output_type=%s&format=json&frame=object' %(
        experiment_id,file_format,output_type
    )
    headers = {'accept': 'application/json'}  #强制返回json数据
    r=requests.get(url, headers=headers).json()

    #从返回的json数据中提取文件属性
    graphs=[]
    for item in r['@graph']:
        prop={}
        prop['experiment']=experiment_id
        prop['assay']=experiment_assay
        prop['target_of_assay']=target
        prop['biosample']=biosample
        prop['accession']=item['accession']
        prop['file_type']=item['file_type']
        prop['file_size']=item['file_size']
        prop['href']=item['href']
        prop['md5sum']=item['md5sum']
        prop['status']=item['status']
        prop['assembly']=item['assembly']
        prop['date_created']=item['date_created']
        try:
            prop['preferred_default']=item['preferred_default']  #ENCODE中带星标的文件
        except:
            prop['preferred_default']='#'
        graphs.append(prop)
    return graphs

def writerx(content_list,to_file):
    with open(to_file, 'w', newline='') as csvfile:
        fieldnames = [
            'experiment',
            'assay',
            'target_of_assay',
            'biosample',
            'accession',
            'file_type',
            'file_size',
            'href',
            'md5sum',
            'status',
            'assembly',
            'date_created',
            'preferred_default']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(content_list)

In [ ]:
#
experiments_file='experiment_report_human_keep.tsv'
assays_file='released_files_of_assays.csv'
log_file='released_files_of_assays.log'

experiments=[]
with open(experiments_file,'r') as f:
    next(f)  #skip first line
    for line in f:
        line=line.strip().split('\t')
        #Accession, Assay name, Target of assay, Biosample term name, Biosample classification
        experiments.append((line[1],line[2],line[6],line[9]))
print(experiments[0])

files=[]
log=open(log_file,'a')  #将日志输出到文件
for exp in experiments:
    print("##### searching...", exp[0],exp[1],exp[2],exp[3],file=log)
    try:
        graph=searcher(exp[0],exp[1],exp[2],exp[3])
        files.extend(graph)
    except:
        print("##### something wrong!", exp[0],exp[1],exp[2],exp[3],file=log)
        pass
log.close()
writerx(files,assays_file)

In [ ]:
# #第一次搜索文件出错的assays，重新搜索，再合并文件直到完整
# #查缺
# experiments_add=[]
# with open(log_file,'r') as f:
#     for line in f:
#         if 'wrong' in line:
#             line=line.split()
#             experiments_add.append(line[3])

# experiments_add=info_human_keep[info_human_keep['Accession'].isin(experiments_add)]
# experiments_add=experiments_add.iloc[:,[1,2,6,9,3]]

# assays_file_add='released_files_of_assays_add.csv'
# #搜索
# files_add=[]
# for i in range(experiments_add.shape[0]):
#     print("##### searching...", experiments_add.iloc[i][0],experiments_add.iloc[i][1],experiments_add.iloc[i][2],experiments_add.iloc[i][3])
#     graph=searcher(experiments_add.iloc[i][0],experiments_add.iloc[i][1],experiments_add.iloc[i][2],experiments_add.iloc[i][3])
#     files_add.extend(graph)
# writerx(files_add,assays_file_add)
# #合并
# assays_1=pd.read_csv(assays_file)
# assays_2=pd.read_csv(assays_file_add)
# assays_merge=pd.concat([assays_1,assays_2])
# print(assays_merge.shape)

In [ ]:
#过滤文件
def get_newest(df,by_='year_created'):
    df = df.sort_values(by = by_,ascending=True)
    return df.iloc[-1,:]

assays_fetch=assays_merge[assays_merge['assembly']=='GRCh38']  #refrence genome 
assays_fetch =assays_fetch[assays_fetch['status']=='released']  #released file
assays_fetch=assays_fetch[assays_fetch['preferred_default']=='True']  #星标的文件
assays_fetch['year_created']=pd.to_datetime(assays_fetch['date_created']).apply(lambda x:int(x.year))

#对于一个实验含有多个带星标文件的情况，选择最新的文件（ENCODE4）
assays_fetch_newest=assays_fetch.groupby(['experiment','assay','biosample'],as_index=False).apply(get_newest)
assays_fetch_newest=assays_fetch_newest.sort_values(by=['biosample','assay'])

print(assays_fetch_newest.shape)
assays_fetch_file='released_files_of_assays_default_samples.csv'
assays_fetch_newest.to_csv(assays_fetch_file,index=False)


In [ ]:
#下载文件
#
import time
from multiprocessing import Pool
import itertools
from tqdm import tqdm

In [ ]:
#
def downloader(url):
    filename=url.split('/')[-1]
    to_file=os.path.join(to_path,filename)

    r = requests.get(url)
    file_size=int(r.headers.get('Content-Length'))
    if os.path.exists(to_file):
        first_byte = os.path.getsize(to_file)
    else:
        first_byte = 0
    if first_byte >= file_size:  #下载完整检验
        print("##### file already exists",filename)
        return file_size

    # with open(to_file, 'wb') as fd:
    #     for chunk in r.iter_content(chunk_size=128):
    #         fd.write(chunk)
    header = {"Range": "bytes=%s-%s" % (first_byte, file_size)}
    pbar = tqdm(
        total=file_size, initial=first_byte,
        unit='B', unit_scale=True, desc=url.split('/')[-1])

    try:
        req = requests.get(url, headers=header, stream=True)
        with(open(to_file, 'ab')) as f:
            for chunk in req.iter_content(chunk_size=128):
                if chunk:
                    f.write(chunk)
                    pbar.update(128)
    except Exception as e:
        print(e)
        return False
    pbar.close()
    return True

In [ ]:
queue_hrefs=assays_fetch_newest['href']
queue_urls=[url_org+x for x in queue_hrefs]

to_path='encode/rawdata'
# for u in queue_urls:
#     downloader(u)
with Pool(processes=12) as pool:  #多线程
    pool.map(downloader,queue_urls)

In [ ]:
#检验文件完整性，md5
!md5sum encode/rawdata/* >encode/rawdata/md5sum.txt

In [ ]:
md5_df=pd.read_table('encode/rawdata/md5sum.txt',sep='\s+',names=['md5sumx','files'])
md5_df['accession']=md5_df['files'].str.split('/',expand=True).iloc[:,3].str.split('.',expand=True).iloc[:,0]

res=pd.merge(assays_fetch_newest[['accession','md5sum','href']],md5_df,on=['accession'],how='left')
files_imcomplete=res[res['md5sum']!=res['md5sumx']]

In [ ]:
#下载不完整的文件重新下载，重新校验
add_hrefs=files_imcomplete['href']
add_urls=[url_org+x for x in add_hrefs]

to_path='../encode/rawdata'
for u in add_urls:
    downloader(u)

#### 根据需要选择细胞系、转录因子和表观数据

相关文件：  
1. ENCODE全部星标文件，released_files_of_assays_default_samples.csv
2. 选择细胞系、转录因子，添加统计信息（peak数量等），released_files_of_assays_default_samples_keep.csv
3. 手动选择文件，用于模型训练，released_files_of_assays_default_samples_keep_select.tsv

In [ ]:
assays_file='released_files_of_assays_default_samples.csv'
assays=pd.read_csv(assays_file)
print(assays.shape)

file_path=assays['accession'].tolist()
file_path=[path_rawdata+x+'.bed.gz' for x in file_path]
assays['file_path']=file_path

assays.head()

In [ ]:
#选择细胞系和表观数据
cells = ['GM12878', 'HCT116', 'HepG2', 'IMR-90', 'K562', 'MCF-7', 'SK-N-SH'] 
chromstates=['ATAC-seq',]
modifications=['H3K27ac','H3K27me3','H3K36me3','H3K4me1','H3K4me3','H3K9me3']

#选择转录因子
# tranfs
tranfs_1=['ATF3', 'BHLHE40', 'CEBPB', 'CTCF', 'EGR1', 'ELF1', 'ELK1', 'FOXM1', 'GABPA', 'JUND', 'MAFK', 'MYC', 'NRF1', 'REST', 'RFX5', 'SP1', 'SRF', 'TCF7L2', 'USF1', 'USF2', 'YY1', 'ZBTB33', 'ZBTB40', 'ZNF24', 'ZNF274', 'CREB1', 'FOS', 'FOXK2', 'MAX', 'MAZ', 'MXI1', 'TCF12', 'TEAD4',]
tranfs_2=['CHD1', 'CHD2', 'EP300', 'HDAC2', 'POLR2A', 'RAD21', 'RCOR1', 'SIN3A', 'SMC3', 'TAF1', 'TARDBP',]
tranfs=tranfs_1+tranfs_2
print('1类数量：',len(tranfs_1))
print('2类数量：',len(tranfs_2))

In [ ]:
assays_state=assays_cell[assays_cell['assay'].isin(chromstates)]
assays_target=assays_cell[assays_cell['target_of_assay'].isin(tranfs+modifications)]
assays_keep=pd.concat([assays_state,assays_target]).reset_index(drop=True)
print(assays_keep.shape)
assays_keep.head()

#### peak统计信息

In [ ]:
#统计peak数量，初始文件数量和过滤后数量
import pyranges as pr

def stat_peak(peak_file):
    acc=peak_file.split('/')[-1][0:11]
    peak=pr.read_bed(peak_file)
    lens=peak.lengths()
    stats=lens.describe()
    lens.acc=acc
    stats.acc=acc
    return lens,stats

#example
_,stats=stat_peak('ENCFF470YYO.bed.gz')
stats

In [ ]:
import math

stats_list=[]
for peakfile in list(assays_keep['file_path']):
    _,stats=stat_peak(peakfile)
    stats_list.append([math.ceil(x*100)/100 for x in list(stats)])


In [ ]:
#
assays_keep['count']=[x[0] for x in stats_list]
assays_keep['mean']=[x[1] for x in stats_list]
assays_keep['std']=[x[2] for x in stats_list]
assays_keep['min']=[x[3] for x in stats_list]
assays_keep['Q1']=[x[4] for x in stats_list]
assays_keep['Q2']=[x[5] for x in stats_list]
assays_keep['Q3']=[x[6] for x in stats_list]
assays_keep['max']=[x[7] for x in stats_list]

file_keep='released_files_of_assays_default_samples_keep.csv'
assays_keep.to_csv(file_keep,index=False)
assays_keep.head()


In [ ]:
#手动挑选peak文件
#这里注意，有的细胞系经过了稳定的基因变异处理，而非原始细胞系，相关数据不采用，如Homo sapiens SK-N-SH genetically modified (insertion) using CRISPR targeting H. sapiens GATA2
#
assays_select_file='released_files_of_assays_default_samples_keep_select.tsv'
assays_select=pd.read_csv(assays_select_file,sep='\t')
print(assays_select.shape)
assays_select.head()

In [ ]:
tranfs_sel=set(list(assays_select['target_of_assay']))
# tranfs_sel
tranfs_sel_1=sorted([x for x in tranfs_sel if x in tranfs_1])
tranfs_sel_2=sorted([x for x in tranfs_sel if x in tranfs_2])
# tranfs_sel_2
#
tranfs_sel=tranfs_sel_1+tranfs_sel_2
print('1类数量：',len(tranfs_sel_1))
print('2类数量：',len(tranfs_sel_2))

In [ ]:
import shutil

peak_path=Path('encode/keepdata')  #data directory
logfile=peak_path/'log.txt'
log=open(logfile,'a')

cells=['K562', 'HepG2', 'GM12878', 'MCF-7', 'SK-N-SH', 'HCT116', 'IMR-90']
for cell in cells:
    peak_path_cell=peak_path/cell
    peak_path_cell.mkdir(exist_ok=True)
    
    cell_info=assays_select[assays_select['biosample']==cell]
    #ATAC-seq改名
    file_sel=cell_info[cell_info['assay']=='ATAC-seq']
    file_from=file_sel['file_path'].tolist()[0]
    file_to=peak_path_cell/'ATAC.bed.gz'
    print(file_from,'-->',file_to,file=log)
    shutil.copy(file_from,file_to)
    
    #histone modifications & TFs
    for ta in modifications+tranfs_sel:
        file_sel=cell_info[cell_info['target_of_assay']==ta]
        try:
            file_from=file_sel['file_path'].tolist()[0]
            file_to=peak_path_cell/(ta+'.bed.gz')
            print(file_from,'-->',file_to,file=log)
            shutil.copy(file_from,file_to)
        except:
            print('no target:',ta,cell,file=log)
log.close()
print('done.')

### FIMO 软件预测转录因子结合位置并与ChIP-peak比较

- 预测

1. 安装[FIMO](https://meme-suite.org/meme/doc/fimo.html?man_type=web)软件并准备相关运行文件
2. 从[JASPAR](https://jaspar.elixir.no/)下载转录因子的motif文件，meme格式
3. 运行

```sh
#!/bin/bash
work_dir='encode/keepdata'

tf_list=(ATF3 BHLHE40 CEBPB CTCF EGR1 ELF1 ELK1 FOXM1 GABPA JUND MAFK MYC NRF1 REST RFX5 SP1 SRF TCF7L2 USF1 USF2 YY1 ZBTB33 ZBTB40 ZNF24 ZNF274 CREB1 FOS FOXK2 MAX MAZ MXI1 TCF12 TEAD4)

log=${work_dir}/fimo_run.log

for tf in ${tf_list[@]}; do
  echo '###'$tf running ... >> ${log};
  meme/bin/fimo --bfile ${work_dir}/fimo/GRCh38.primary_assembly.genome.fa.bfile --thresh 1e-4 --max-stored-scores 100000 --o ${work_dir}/fimo/${tf} ${work_dir}/fimo/meme/${tf}_*.meme genome/gencode/GRCh38.primary_assembly.genome.fa &;
  mv ${work_dir}/fimo/${tf}/fimo.gff ${work_dir}/fimo/${tf}.gff;
done
```

- 与ChIP-peak的比较

```sh
#!/bin/bash
work_dir='encode/keepdata'
tf_list=(ATF3 BHLHE40 CEBPB CTCF EGR1 ELF1 ELK1 FOXM1 GABPA JUND MAFK MYC NRF1 REST RFX5 SP1 SRF TCF7L2 USF1 USF2 YY1 ZBTB33 ZBTB40 ZNF24 ZNF274 CREB1 FOS FOXK2 MAX MAZ MXI1 TCF12 TEAD4)
ce_list=(K562 HepG2 GM12878 MCF-7 SK-N-SH HCT116 IMR-90)

log=${work_dir}/fimo_compare.log

for tf in ${tf_list[@]}; do
  echo '###'${tf} `wc -l ${work_dir}/fimo/${tf}.gff`;
  echo '###'${tf} `wc -l ${work_dir}/fimo/${tf}.gff` >> ${log};
  for ce in ${ce_list[@]}; do
    echo '###'${ce} >> ${log};
    fn=${work_dir}/${ce}/${tf}.bed;
    if [ -f ${fn} ]; then
      mkdir ${work_dir}/${ce}/${tf};
      bedtools intersect -a ${work_dir}/fimo/${tf}.gff -b ${fn} -v |awk -F '\t' -v OFS='\t' '{print $1,$4,$5,$9,$6,$7}'> ${work_dir}/${ce}/${tf}/fimo_neg.bed;
      sleep 10;
      bedtools intersect -a ${work_dir}/fimo/${tf}.gff -b ${fn} -u |awk -F '\t' -v OFS='\t' '{print $1,$4,$5,$9,$6,$7}'> ${work_dir}/${ce}/${tf}/fimo_pos.bed;
      sleep 10;
      echo fimo_neg.bed `wc -l ${work_dir}/encode/${ce}/${tf}/fimo_neg.bed` >> ${log};
      echo fimo_pos.bed `wc -l ${work_dir}/encode/${ce}/${tf}/fimo_pos.bed` >> ${log};
    fi
  done
done
```

*使用bedtools window*

```
#log=${work_dir}/endchip/fimo2_w.log
#mkdir ${work_dir}/encode/${ce}/${tf}_w;
#bedtools window -a ${work_dir}/fimo/${tf}.gff -b ${fn} -w 200 -v |awk -F '\t' -v OFS='\t' '{print $1,$4,$5,$9,$6,$7}'> ${work_dir}/${ce}/${tf}_w/fimo_neg.bed;
#bedtools window -a ${work_dir}/fimo/${tf}.gff -b ${fn} -w 200 -u |awk -F '\t' -v OFS='\t' '{print $1,$4,$5,$9,$6,$7}'> ${work_dir}/${ce}/${tf}_w/fimo_pos.bed;
#echo fimo_neg.bed `wc -l ${work_dir}/${ce}/${tf}_w/fimo_neg.bed` >> ${log};
#echo fimo_pos.bed `wc -l ${work_dir}/${ce}/${tf}_w/fimo_pos.bed` >> ${log};
```
